In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
%matplotlib inline
from keras.layers import Input, Dense
from keras.models import Model
# from keras.op
from keras import optimizers
from keras import regularizers
from PIL import Image
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

Using TensorFlow backend.


In [2]:
test_percent = 0.95
test_percent_str = "95"
images_path = "C:/Users/Ghevond/Desktop/Specs5/"
a_list = []
data_train_names = os.listdir(images_path)
pattern = "*.tiff"
for entry in data_train_names:
    if fnmatch.fnmatch(entry, pattern):
            a_list.append(images_path + entry)

In [3]:
print(len(a_list))

1132796


In [4]:
def split_list(the_list, chunk_size):
    result_list = []
    while the_list:
        result_list.append(the_list[:chunk_size])
        the_list = the_list[chunk_size:]
    return result_list

list_of_chunks = split_list(a_list, 25000)

In [5]:
print(len(list_of_chunks))

46


In [6]:
from tensorflow.python.client import device_lib 
import tensorflow as tf 
import keras

#print(device_lib.list_local_devices())

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [7]:
from keras.models import load_model
from keras.models import model_from_json
import json

with open('C:/Users/Ghevond/Desktop/Astro/models/4/deep_model_16bit_07~02~2020.json','r') as f:
    model_json = json.load(f)

s1 = json.dumps(model_json)
    
model = model_from_json(s1)
model.load_weights('C:/Users/Ghevond/Desktop/Astro/models/4/deep_model_16bit_07~02~2020.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






In [8]:
for i_n_d_e_x in range(3, len(list_of_chunks)):
    print("----------------BEGIN-----------------")
    images_data = list_of_chunks[i_n_d_e_x]
    print(len(images_data))
    max_width = 50
    max_height = 160
    images_list = []
    
    ind = 0
    for i in range(len(images_data)):
        im = Image.open(images_data[i])
        arr = np.array(im)#.flatten()
    
        arr=(arr-arr.min())/(arr.max()-arr.min())
        #arr = arr / 0xffff
        images_list.append(arr)
        
    for i in range(len(images_list)):
        s = images_list[i].shape
        d_width = (max_width - s[1])
        d_height = (max_height - s[0])
        
        d_top = int(d_height / 2)
        d_bottom = int(d_height - d_top)
        
        d_left = int(d_width / 2)
        d_right = int(d_width - d_left)
        #print(d_top, d_bottom, d_left, d_right)
        
        arr = images_list[i]
        for l in range(d_left):
            arr = np.insert(arr, 0, 0, axis = 1)
        
        for r in range(d_right):
            b = np.zeros((s[0],1))
            arr = np.append(arr, b, axis = 1)
        
        for t in range(d_top):
            arr = np.insert(arr, 0, 0, axis = 0)
        
        for b in range(d_bottom):
            b = np.zeros((1, arr.shape[1],))
            arr = np.append(arr, b, axis = 0)
        
        images_list[i] = arr.flatten()
        
    width = max_width
    height = max_height
    #print(width, height)
    #plt.imshow(images_list[0].reshape(140, 40))
    
    images_np = np.array(images_list)
    print(images_np.shape)
    images_np = images_np.reshape(images_np.shape[0], max_height, max_width, 1)
    print(images_np.shape)
    
    print("--------------PREDICTION-------------------------")
    arr_res = model.predict(images_np)
    
    arr_class = []
    for i1 in range(len(arr_res)):
        max_p = np.max(arr_res[i1])
        res_class = -1;
        if(max_p >= test_percent):
            res_class = np.argmax(arr_res[i1])
    
        arr_class.append(res_class)
        
    print("--------------SAVEING-------------------------")
    out_path = "C:/Users/Ghevond/Desktop/Astro/New/Test"+ test_percent_str +"/"
    for i2 in range(len(arr_class)):
        current_image = images_data[i2]
        class_string = str(arr_class[i2])
        if(class_string == "-1"): 
            continue
        new_path = out_path + class_string + "/" + current_image.split("/")[5]
        #print(current_image, class_string, new_path)
        #break
        if(class_string == "3"):
            im = Image.open(current_image)
            im.save(new_path)
        else:
            image_file_name = current_image.split("/")[5]
            res_file_path = out_path + class_string + "/result.txt"
            outF = open(res_file_path, "a")
            outF.write(image_file_name + "\n")
            #outF.write("\n")
            outF.close()
        
    print("-------------END["+ str(i_n_d_e_x) +"]----------------------")

----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[3]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[4]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[5]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[6]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------

--------------SAVEING-------------------------
-------------END[40]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[41]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[42]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[43]----------------------
----------------BEGIN-----------------
25000
(25000, 8000)
(25000, 160, 50, 1)
--------------PREDICTION-------------------------
--------------SAVEING-------------------------
-------------END[44]----------------------
----------------BEGIN-------------